In [4]:
import pandas as pd
import numpy as np
import networkx as nx

df_cleaning = pd.read_excel('fin_cleansing.xlsx', 'Sheet1')
df = pd.read_excel('poc_fin_masked_lama.xlsx', 'Searah')
df_checkpoint = pd.read_excel('poc_fin_masked_lama.xlsx', 'Checkpoint')
df_uplink = pd.read_excel('uplink_ori_dummy.xlsx', 'Sheet2')
df_downlink = pd.read_excel('downlink_ori_dummy.xlsx', 'Sheet1')

In [5]:
#shortestpath uplink HSI
H = nx.DiGraph()

for i in range(len(df)):
  from1_element = df.iloc[i]['from1']
  to1_element = df.iloc[i]['to1']
  cost = df.iloc[i]['cost_link']
  if 'VPN' in from1_element or'VPN' in to1_element or 'TRANSIT' in from1_element or'TRANSIT' in to1_element:
    continue
#   elif 'MET9**KBL' in from1_element and 'MET9**RKT' in to1_element:
#     continue
#   elif 'MET9**RKT' in from1_element and 'MET9**KBL' in to1_element:
#     continue
  else:
    H.add_edge(from1_element,to1_element,weight=cost)

  #print("from: ", from1_element, "to: ", to1_element, "cost: ",cost)


def remove_edge(H, node_from, node_to):
    if H.has_edge(node_from, node_to):
        H.remove_edge(node_from, node_to)

for i in range(len(df_checkpoint)):
    for j in range(len(df_checkpoint)):
        remove_edge(H, df_checkpoint.iloc[i]['Checkpoint'], df_checkpoint.iloc[j-1]['Checkpoint'])


# shortest_path = nx.shortest_path(H, 'MET9**KBL', 'MET9**RKT', weight='weight')
# length = nx.shortest_path_length(H, 'OLT00**ABU*2DSU', 'MET9**KBL', weight='weight')
# print("Shortest Path:", shortest_path)
# print("total_cost:", length)

In [6]:
#shortestpath uplink VPN
V = nx.DiGraph()

for i in range(len(df)):
  from1_element = df.iloc[i]['from1']
  to1_element = df.iloc[i]['to1']
  cost = df.iloc[i]['cost_link']
  if 'BR' in from1_element or 'BR' in to1_element:
    continue
#   elif 'MET9**KBL' in from1_element and 'MET9**RKT' in to1_element:
#     continue
#   elif 'MET9**RKT' in from1_element and 'MET9**KBL' in to1_element:
#     continue
  else:
    V.add_edge(from1_element,to1_element,weight=cost)

  #print("from: ", from1_element, "to: ", to1_element, "cost: ",cost)

shortest_path = nx.shortest_path(V, 'MET9**RKT', 'CNFacebook', weight='weight')
length = nx.shortest_path_length(V, 'OLT00**ABU*2DSU', 'MET9**KBL', weight='weight')
print("Shortest Path:", shortest_path)
print("total_cost:", length)

Shortest Path: ['MET9**RKT', 'PE2**RKT*VPN', 'P**RKT', 'P**KBL', 'PE**KBL*VPN', 'MET4**KBL', 'PE**KBL*HSI', 'CNFacebook']
total_cost: 300


In [7]:
#shortestpath uplink Transit
T = nx.DiGraph()

for i in range(len(df)):
  from1_element = df.iloc[i]['from1']
  to1_element = df.iloc[i]['to1']
  cost = df.iloc[i]['cost_link']
  if 'BR' in from1_element or 'BR' in to1_element or 'VPN' in from1_element or'VPN' in to1_element:
    continue
#   elif 'MET9**KBL' in from1_element and 'MET9**RKT' in to1_element:
#     continue
#   elif 'MET9**RKT' in from1_element and 'MET9**KBL' in to1_element:
#     continue
  else:
    T.add_edge(from1_element,to1_element,weight=cost)

  #print("from: ", from1_element, "to: ", to1_element, "cost: ",cost)

shortest_path = nx.shortest_path(T, 'MET9**RKT', 'CGW*.PNK', weight='weight')
length = nx.shortest_path_length(T, 'OLT00**ABU*2DSU', 'MET9**KBL', weight='weight')
print("Shortest Path:", shortest_path)
print("total_cost:", length)

Shortest Path: ['MET9**RKT', 'PE2**RKT*TRANSIT', 'P**RKT', 'P*D6*ULN', 'P*D7*PNK', 'CGW*.PNK']
total_cost: 300


In [8]:
path_hsi = []
total_cost_hsi = []
checkpoint_hsi = []

path_vpn = []
total_cost_vpn = []

path_transit = []
total_cost_transit = []

def check_path_exists(H, node_from, node_to):
    return nx.has_path(H, node_from, node_to)

for i in range(len(df_uplink)):
    from1 = df_uplink.iloc[i]['olt1']
    to1 = df_uplink.iloc[i]['to1']
    produk = df_uplink.iloc[i]['produk']
    metro = 999999
    if produk == 'hsi':
        for chk in range(len(df_checkpoint)):
            if check_path_exists(H, from1, df_checkpoint.iloc[chk]['Checkpoint']):
                if nx.shortest_path_length(H, from1, df_checkpoint.iloc[chk]['Checkpoint'], weight='weight') < metro:
                    metro = nx.shortest_path_length(H, from1, df_checkpoint.iloc[chk]['Checkpoint'], weight='weight')
                    checkpoint = df_checkpoint.iloc[chk]['Checkpoint']
            else:
                continue       
        path_hsi.append(nx.shortest_path(H, from1, checkpoint, weight='weight'))
        total_cost_hsi.append(metro)
        checkpoint_hsi.append(checkpoint)

    elif produk == 'vpn':
        path_vpn.append(nx.shortest_path(V, from1,to1, weight='weight'))
        total_cost_vpn.append(nx.shortest_path_length(V, from1, to1, weight='weight'))
        
    elif produk == 'transit':
        path_transit.append(nx.shortest_path(T, from1,to1, weight='weight'))
        total_cost_transit.append(nx.shortest_path_length(T, from1, to1, weight='weight'))

df_baru_hsi = df_uplink[df_uplink['produk'] == 'hsi']
df_baru_hsi = df_baru_hsi.drop(columns=['Unnamed: 0'])
df_baru_hsi['path'] = path_hsi
df_baru_hsi['total_cost'] = total_cost_hsi
df_baru_hsi['checkpoint'] = checkpoint_hsi

df_baru_vpn = df_uplink[df_uplink['produk'] == 'vpn']
df_baru_vpn = df_baru_vpn.drop(columns=['Unnamed: 0'])
df_baru_vpn['path'] = path_vpn
df_baru_vpn['total_cost'] = total_cost_vpn

df_baru_transit = df_uplink[df_uplink['produk'] == 'transit']
df_baru_transit = df_baru_transit.drop(columns=['Unnamed: 0'])
df_baru_transit['path'] = path_transit
df_baru_transit['total_cost'] = total_cost_transit

In [9]:
path = []
costs = []
for i in range(len(df_baru_hsi)):
    sp = nx.shortest_path(H, df_baru_hsi.iloc[i]['checkpoint'], df_baru_hsi.iloc[i]['to1'] , weight='weight')
    tc = nx.shortest_path_length(H, df_baru_hsi.iloc[i]['checkpoint'], df_baru_hsi.iloc[i]['to1'] , weight='weight')
    sp.pop(0)
    
    path.append(df_baru_hsi.iloc[i]['path'] + sp)
    costs.append(df_baru_hsi.iloc[i]['total_cost'] + tc)
    
df_akhir_hsi = df_baru_hsi.copy() 
df_akhir_hsi = df_akhir_hsi.drop('path', axis=1)
df_akhir_hsi = df_akhir_hsi.drop('total_cost', axis=1)
df_akhir_hsi = df_akhir_hsi.drop('checkpoint', axis=1)
df_akhir_hsi['path'] = path
df_akhir_hsi['total_cost'] = costs

df_combined_uplink = pd.concat([df_akhir_hsi, df_baru_vpn, df_baru_transit])
df_combined_uplink

,olt1,to1,service,bw_up,produk,path,total_cost
0,MOLT00**MGO*3GDH,CNFacebook,MOBILE,0.004000,hsi,"[MOLT00**MGO*3GDH, MET**DMO, MET9**RKT, BR2**R...",60010
1,MOLT01**MGO*3TP6,CNFacebook,MOBILE,0.002000,hsi,"[MOLT01**MGO*3TP6, MET**DMO, MET9**RKT, BR2**R...",60010
2,OLT01**KNN*3,CNFacebook,MOBILE,0.041000,hsi,"[OLT01**KNN*3, MET**KNN, MET9**KBL, BR2**KBL, ...",65000
3,OLT00**KDE*2BRJ,CNFacebook,MOBILE,0.002000,hsi,"[OLT00**KDE*2BRJ, MET**KDM, MET2**LKS, MET2**K...",65030
4,OLT03**KRP*2,CGW*.MDO,MOBILE,0.035000,hsi,"[OLT03**KRP*2, MET2**KRP, MET9**RKT, BR2**RKT,...",65809
...,...,...,...,...,...,...,...
9896,MET8**RKT,CNHuawei,EBIS,0.020775,transit,"[MET8**RKT, PE2**RKT*TRANSIT, P**RKT, P**KBL, ...",90000
9897,MET8**RKT,CNFacebook,MOBILE,0.049664,transit,"[MET8**RKT, PE2**RKT*TRANSIT, P**RKT, P**KBL, ...",30010
9898,MET8**RKT,CNFacebook,RETAIL,0.039616,transit,"[MET8**RKT, PE2**RKT*TRANSIT, P**RKT, P**KBL, ...",30010
9899,MET8**RKT,CGW*.PNK,MOBILE,0.019403,transit,"[MET8**RKT, PE2**RKT*TRANSIT, P**RKT, P*D6*ULN...",30809


In [10]:
path_hsi = []
total_cost_hsi = []
checkpoint_hsi = []

path_vpn = []
total_cost_vpn = []

path_transit = []
total_cost_transit = []

def check_path_exists(H, node_from, node_to):
    return nx.has_path(H, node_from, node_to)

for i in range(len(df_downlink)):
    from1 = df_downlink.iloc[i]['olt1']
    to1 = df_downlink.iloc[i]['from1']
    produk = df_downlink.iloc[i]['produk']
    metro = 999999
    if produk == 'hsi':
        for chk in range(len(df_checkpoint)):
            if check_path_exists(H, from1, df_checkpoint.iloc[chk]['Checkpoint']):
                if nx.shortest_path_length(H, from1, df_checkpoint.iloc[chk]['Checkpoint'], weight='weight') < metro:
                    metro = nx.shortest_path_length(H, from1, df_checkpoint.iloc[chk]['Checkpoint'], weight='weight')
                    checkpoint = df_checkpoint.iloc[chk]['Checkpoint']
            else:
                continue       
        path_hsi.append(nx.shortest_path(H, from1, checkpoint, weight='weight'))
        total_cost_hsi.append(metro)
        checkpoint_hsi.append(checkpoint)

    elif produk == 'vpn':
        path_vpn.append(nx.shortest_path(V, from1,to1, weight='weight'))
        total_cost_vpn.append(nx.shortest_path_length(V, from1, to1, weight='weight'))
        
    elif produk == 'transit':
        path_transit.append(nx.shortest_path(T, from1,to1, weight='weight'))
        total_cost_transit.append(nx.shortest_path_length(T, from1, to1, weight='weight'))

df_baru_hsi = df_downlink[df_downlink['produk'] == 'hsi']
df_baru_hsi = df_baru_hsi.drop(columns=['Unnamed: 0'])
df_baru_hsi['path'] = path_hsi
df_baru_hsi['total_cost'] = total_cost_hsi
df_baru_hsi['checkpoint'] = checkpoint_hsi


df_baru_vpn = df_downlink[df_downlink['produk'] == 'vpn']
df_baru_vpn = df_baru_vpn.drop(columns=['Unnamed: 0'])
df_baru_vpn['path'] = path_vpn
df_baru_vpn['total_cost'] = total_cost_vpn

df_baru_transit = df_downlink[df_downlink['produk'] == 'transit']
df_baru_transit = df_baru_transit.drop(columns=['Unnamed: 0'])
df_baru_transit['path'] = path_transit
df_baru_transit['total_cost'] = total_cost_transit

df_baru_hsi

,from1,olt1,service,bw_down,produk,path,total_cost,checkpoint
0,CGW*.BDS,MOLT01**MGO*3TP6,MOBILE,0.012,hsi,"[MOLT01**MGO*3TP6, MET**DMO, MET9**RKT]",60000,MET9**RKT
1,CNConversant,OLT02**ABU*2,MOBILE,0.020,hsi,"[OLT02**ABU*2, MET**ABT, MET2**SMP, MET9**RKT]",200,MET9**RKT
2,CNFacebook,OLT12**DMO*3,MOBILE,0.218,hsi,"[OLT12**DMO*3, MET**DMO, MET9**RKT]",60000,MET9**RKT
3,CNNetflix,OLT12**DMO*3,MOBILE,0.218,hsi,"[OLT12**DMO*3, MET**DMO, MET9**RKT]",60000,MET9**RKT
4,CNFacebook,OLT00**MGO*2WDR,MOBILE,0.000,hsi,"[OLT00**MGO*2WDR, MET**DMO, MET9**RKT]",60000,MET9**RKT
...,...,...,...,...,...,...,...,...
4945,CGW2*.BTC,OLT01**ABU*3,RETAIL,0.018,hsi,"[OLT01**ABU*3, MET**ABT, MET2**SMP, MET9**RKT]",200,MET9**RKT
4946,TEL#_CGW2*.BDS,OLT01**ABU*3,RETAIL,0.018,hsi,"[OLT01**ABU*3, MET**ABT, MET2**SMP, MET9**RKT]",200,MET9**RKT
4947,CGW2*.BTC,OLT01**SMP*3,RETAIL,0.091,hsi,"[OLT01**SMP*3, MET2**SMP, MET9**RKT]",100,MET9**RKT
4948,CGW2*.BTC,OLT01**SMP*3,RETAIL,0.091,hsi,"[OLT01**SMP*3, MET2**SMP, MET9**RKT]",100,MET9**RKT


In [11]:
#DOWNLINK
path = []
costs = []
for i in range(len(df_baru_hsi)):
    sp = nx.shortest_path(H, df_baru_hsi.iloc[i]['checkpoint'], df_baru_hsi.iloc[i]['from1'] , weight='weight')
    tc = nx.shortest_path_length(H, df_baru_hsi.iloc[i]['checkpoint'], df_baru_hsi.iloc[i]['from1'] , weight='weight')
    sp.pop(0)
    
    path.append(df_baru_hsi.iloc[i]['path'] + sp)
    costs.append(df_baru_hsi.iloc[i]['total_cost'] + tc)
    
df_akhir_hsi = df_baru_hsi.copy() 
df_akhir_hsi = df_akhir_hsi.drop('path', axis=1)
df_akhir_hsi = df_akhir_hsi.drop('total_cost', axis=1)
df_akhir_hsi = df_akhir_hsi.drop('checkpoint', axis=1)
df_akhir_hsi['path'] = path
df_akhir_hsi['total_cost'] = costs

df_combined_downlink = pd.concat([df_akhir_hsi, df_baru_vpn, df_baru_transit])

path = []
for i in range(len(df_combined_downlink)):
    shortest_path = df_combined_downlink.iloc[i]['path']
    shortest_path = shortest_path[::-1]
    path.append(shortest_path)
    
df_combined_downlink['path'] = path

In [12]:
df_cleaning = df_cleaning.drop_duplicates()

df_cleaning = df_cleaning.rename({'from1': 'node', 'to1': 'neighbor'}, axis='columns')

# Membuat kolom kosong untuk menyimpan besaran total traffic yang melewati tiap edge
df_cleaning['inbound'] = 0
df_cleaning['outbound'] = 0
df_cleaning['dilewati'] = False
df_cleaning['status inbound'] = 'aman'
df_cleaning['status outbound'] = 'aman'
df_cleaning['total_loss_inbound'] = 0
df_cleaning['total_loss_outbound'] = 0

df_cleaning

,node,neighbor,kap,jmllink,cost_link,inbound,outbound,dilewati,status inbound,status outbound,total_loss_inbound,total_loss_outbound
0,BR2**KBL,PE**KBL*HSI,520,7.0,0,0,0,False,aman,aman,0,0
1,BR2**RKT,PE**RKT*HSI,1100,11.0,0,0,0,False,aman,aman,0,0
2,MET2**KBL,PE**KBL*HSI,10,1.0,0,0,0,False,aman,aman,0,0
3,MET3**RKT,PE**RKT*HSI,1,1.0,0,0,0,False,aman,aman,0,0
4,MET4**KBL,PE**KBL*HSI,10,1.0,0,0,0,False,aman,aman,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
467,P*D1*BDS,P**RKT,2600,NaN,0,0,0,False,aman,aman,0,0
468,P*D7*PNK,P**RKT,300,NaN,3096,0,0,False,aman,aman,0,0
469,P*D1*BTC,P*D7*MDO,270,NaN,3000,0,0,False,aman,aman,0,0
470,P**KBL,P*D7*MDO,200,NaN,2895,0,0,False,aman,aman,0,0


In [13]:
# Melakukan injeksi traffic uplink
# Melakukan sort berdasarkan prioritas jenis service
custom_dict = {'MOBILE': 0, 'WHOLESALE': 1, 'EBIS': 2, 'RETAIL': 3}
df_combined_uplink = df_combined_uplink.sort_values(by=['service'], key=lambda x: x.map(custom_dict))
df_combined_uplink = df_combined_uplink.reset_index(drop=True)
df_combined_uplink['bw_final'] = df_combined_uplink['bw_up']

# Iterasi untuk tiap traffic df_uplink
for i in range(len(df_combined_uplink)):

  # Melakukan konversi string shortest_path ke bentuk array
  shortest_path = str(df_combined_uplink.iloc[i]['path'])
  shortest_path = shortest_path[1:-1]
  # print(shortest_path)
  sp_array = list(shortest_path.split(", "))

  # Melakukan iterasi untuk tiap edge di shortest_path
  for j in range(len(sp_array)-1):

    # Mencari index edge yang sesuai di dataframe topologi (fin)
    idx_edge = (df_cleaning.loc[(df_cleaning['node'] == sp_array[j][1:-1])&(df_cleaning['neighbor']==sp_array[j+1][1:-1])].index.values)[0]
    idx_edge2 = (df_cleaning.loc[(df_cleaning['neighbor'] == sp_array[j][1:-1])&(df_cleaning['node']==sp_array[j+1][1:-1])].index.values)[0]
    # print("Edge antara "+ str(sp_array[j][1:-1])+" dan "+ str(sp_array[j+1][1:-1]))
    # print("Punya index: "+str(idx_edge)+"\n")

    # Perhitungan berapa besaran traffic yang bisa masuk ke dalam edge
    max_kap = float(df_cleaning.iloc[idx_edge]['kap'])
    cur_traffic = float(df_cleaning.iloc[idx_edge]['outbound'])
    bw_cur = float(df_combined_uplink.iloc[i]['bw_final'])
    if ((cur_traffic + bw_cur) > max_kap): # Jika jumlah
      df_combined_uplink.at[i, 'bw_final'] = max_kap - cur_traffic
      df_cleaning.at[idx_edge, 'outbound'] += df_combined_uplink.iloc[i]['bw_final']
      df_cleaning.at[idx_edge, 'status outbound'] = 'loss'
      df_cleaning.at[idx_edge,'total_loss_outbound'] += (cur_traffic-(max_kap-bw_cur))
    else:
      df_cleaning.at[idx_edge, 'outbound'] += df_combined_uplink.iloc[i]['bw_final']
    df_cleaning.at[idx_edge,'dilewati'] = True

    max_kap_reverse = float(df_cleaning.iloc[idx_edge2]['kap'])
    cur_traffic_reverse = float(df_cleaning.iloc[idx_edge2]['inbound'])
    bw_cur1 = float(df_combined_uplink.iloc[i]['bw_final'])
    if ((cur_traffic_reverse + bw_cur) > max_kap): # Jika jumlah
      df_combined_uplink.at[i, 'bw_final'] = max_kap - cur_traffic_reverse
      df_cleaning.at[idx_edge2, 'inbound'] += df_combined_uplink.iloc[i]['bw_final']
      df_cleaning.at[idx_edge2, 'status inbound'] = 'loss'
      df_cleaning.at[idx_edge2,'total_loss_inbound'] += (cur_traffic_reverse-(max_kap-bw_cur))
    else:
      df_cleaning.at[idx_edge2, 'inbound'] += df_combined_uplink.iloc[i]['bw_final']
    df_cleaning.at[idx_edge2,'dilewati'] = True

# for i in range(len(df_uplink)):
#   shortest_path = df_uplink.iloc[i]['shortest_path']
#   shortest_path = shortest_path[1:-1]
#   # print(shortest_path)
#   sp_array = list(shortest_path.split(", "))
#   for j in range(len(sp_array)-1):
#     # print(row[1]['olt1'])
#     # print(sp_array[i][1:-1])
#     # print(type(sp_array[i]))
#     idx_edge = (df.loc[(df['from1'] == sp_array[j][1:-1])&(df['to1']==sp_array[j+1][1:-1])].index.values)[0]
#     print("Edge antara "+ str(sp_array[j][1:-1])+" dan "+ str(sp_array[j+1][1:-1]))
#     print("Punya index: "+str(idx_edge))
#     df.at[idx_edge, 'current_kap'] += df_uplink.iloc[i]['bw_up']

#     print("BW terpakai: "+str(df_uplink.iloc[i]['bw_up'])+"\n")

# df_cleaning = df_cleaning[df_cleaning.dilewati == True]
# df_cleaning = df_cleaning.reset_index(drop=True)

In [14]:
df_combined_downlink['bw_final'] = df_combined_downlink['bw_down']
custom_dict = {'MOBILE': 0, 'WHOLESALE': 1, 'EBIS': 2, 'RETAIL': 3}
df_combined_downlink = df_combined_downlink.sort_values(by=['service'], key=lambda x: x.map(custom_dict))
df_combined_downlink = df_combined_downlink.reset_index(drop=True)

for i in range(len(df_combined_downlink)):
  shortest_path = str(df_combined_downlink.iloc[i]['path'])
  shortest_path = shortest_path[1:-1]
  sp_array = list(shortest_path.split(", "))
  # print(sp_array)
  for j in range(len(sp_array)-1):
    # print(row[1]['olt1'])
    # print(sp_array[i][1:-1])
    # print(type(sp_array[i]))
    idx_edge = (df_cleaning.loc[(df_cleaning['node'] == sp_array[j][1:-1])&(df_cleaning['neighbor']==sp_array[j+1][1:-1])].index.values)[0]
    idx_edge2 = (df_cleaning.loc[(df_cleaning['neighbor'] == sp_array[j][1:-1])&(df_cleaning['node']==sp_array[j+1][1:-1])].index.values)[0]
    #print("Edge antara "+ str(sp_array[j][1:-1])+" dan "+ str(sp_array[j+1][1:-1]))
    # print("Punya index: "+str(idx_edge)+"\n")

    max_kap = float(df_cleaning.iloc[idx_edge]['kap'])
    cur_traffic = float(df_cleaning.iloc[idx_edge]['outbound'])
    bw_cur = float(df_combined_downlink.iloc[i]['bw_final'])
    df_cleaning.at[idx_edge,'dilewati'] = True
    if ((cur_traffic + bw_cur) > max_kap): # Jika jumlah
      df_combined_downlink.at[i, 'bw_final'] = max_kap - cur_traffic
      df_cleaning.at[idx_edge, 'outbound'] += df_combined_downlink.iloc[i]['bw_final']
      df_cleaning.at[idx_edge, 'status outbound'] = 'loss'
      df_cleaning.at[idx_edge,'total_loss_outbound'] += (cur_traffic-(max_kap-bw_cur))
    else:
        df_cleaning.at[idx_edge, 'outbound'] += df_combined_downlink.iloc[i]['bw_final']
    
    max_kap_reverse = float(df_cleaning.iloc[idx_edge2]['kap'])
    cur_traffic_reverse = float(df_cleaning.iloc[idx_edge2]['inbound'])
    bw_cur1 = float(df_combined_downlink.iloc[i]['bw_final'])
    df_cleaning.at[idx_edge2,'dilewati'] = True
    if ((cur_traffic_reverse + bw_cur) > max_kap): # Jika jumlah
      df_combined_downlink.at[i, 'bw_final'] = max_kap - cur_traffic_reverse
      df_cleaning.at[idx_edge2, 'inbound'] += df_combined_downlink.iloc[i]['bw_final']
      df_cleaning.at[idx_edge2, 'status inbound'] = 'loss'
      df_cleaning.at[idx_edge2,'total_loss_inbound'] += (cur_traffic_reverse-(max_kap-bw_cur))
    else:
      df_cleaning.at[idx_edge2, 'inbound'] += df_combined_downlink.iloc[i]['bw_final']
        
df_cleaning = df_cleaning[df_cleaning.dilewati == True]
df_cleaning = df_cleaning.reset_index(drop=True)  
df_cleaning

,node,neighbor,kap,jmllink,cost_link,inbound,outbound,dilewati,status inbound,status outbound,total_loss_inbound,total_loss_outbound
0,BR2**KBL,PE**KBL*HSI,520,7.0,0,96.119000,31.496000,True,aman,aman,0.000000,0.000000
1,BR2**RKT,PE**RKT*HSI,1100,11.0,0,191.761000,63.302000,True,aman,aman,0.000000,0.000000
2,MET4**KBL,PE**KBL*HSI,10,1.0,0,10.000000,10.000000,True,loss,loss,25.515119,4.080476
3,OLT00**ABT*3,MET**ABT,1,1.0,0,1.000000,0.790000,True,loss,aman,1.287000,0.000000
4,OLT00**ABU*2DSU,MET**ABT,1,1.0,0,1.000000,0.786000,True,loss,aman,1.282000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
301,MET4**KBL,PE**KBL*VPN,13,NaN,0,10.602293,13.000000,True,aman,loss,0.000000,3.299463
302,P*D1*BTC,P*D1*BDS,3800,NaN,0,23.143549,65.273162,True,aman,aman,0.000000,0.000000
303,P**RKT,P*D1*BDS,2600,NaN,0,250.741310,93.841545,True,aman,aman,0.000000,0.000000
304,P*D1*BDS,P*D1*BTC,3800,NaN,0,65.273162,23.143549,True,aman,aman,0.000000,0.000000


In [15]:
#Inject traffic uplink

import webbrowser
from pyvis import network as net

G = net.Network(notebook=True, directed=True, select_menu=False)

# Menambahkan simpul-simpul dari kolom 'from1'
for i in range(len(df_cleaning)):
    node = df_cleaning.iloc[i]['node']
    if "OLT" in node:
        warna='#FF0000'  # Merah untuk simpul dengan "OLT" di dalamnya
    elif "MET" in node:
        warna='#FFFF00'  # Biru untuk simpul lainnya
    elif "PE" in node or "P*" in node or "T" in node or "P*D1*PBR" in node:
        warna='#0000FF'
    if "BR2**KBL" in node or "BR2**RKT" in node:
        warna='#00FF00'
    if "CN" in node or "CGW" in node or "CGW2*.BTC" in node or "TEL#_CGW" in node:
        warna='#000000'

    G.add_node(node, color=warna)

# Menambahkan simpul-simpul dari kolom 'to1'


# Mengiterasi setiap baris pada dataframe
for i in range(len(df_cleaning)):
    from1_element = df_cleaning.iloc[i]['node']
    to1_element = df_cleaning.iloc[i]['neighbor']
    #cost_link = "Cost link: " + str(df_uplink.iloc[i]['cost_link']) + "\nKapasitas: " + str(df_uplink.iloc[i]['kap']) + "\nOutbound dari " + str(df_uplink.iloc[i]['node']) + " ke " + str(df_uplink.iloc[i]['neighbor']) +" : " + str(df_uplink.iloc[i]['inbound'])+ "\nInbound ke " + str(df_uplink.iloc[i]['neighbor']) + " dari " + str(df_uplink.iloc[i]['neighbor']) +  " : " + str(df_uplink.iloc[i]['inbound'])
    cost_link = "Cost link: " + str(df_cleaning.iloc[i]['cost_link']) + "\nKapasitas: " + str(df_cleaning.iloc[i]['kap']) + "\nKe arah " + str(df_cleaning.iloc[i]['neighbor']) + ": " + str(df_cleaning.iloc[i]['outbound']) + "\nTotal loss: " + str(df_cleaning.iloc[i]['total_loss_outbound'])
    
    if (df_cleaning.iloc[i]['outbound'] > df_cleaning.iloc[i]['kap'] ):
        G.add_edge(from1_element, to1_element, title=cost_link, color='#FF0000')  # Mengatur edge menjadi directed dengan arrows=True
    else:
        G.add_edge(from1_element, to1_element, title=cost_link, color='#000000')
        
    
# Menampilkan graf menggunakan Pyvis
G.show('graph_uplink.html')

webbrowser.open('graph_uplink.html',new=2)

graph_uplink.html


True